In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [14]:
feature_set =  'feature_set_5_normalised'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv')
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

In [15]:
random_state = 42

In [16]:
forest_model = RandomForestRegressor(n_estimators=256,random_state=random_state,verbose=1,n_jobs=-1)
forest_model.fit(X_train,y_train)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 256 out of 256 | elapsed: 13.4min finished


RandomForestRegressor(n_estimators=256, n_jobs=-1, random_state=42, verbose=1)

In [20]:
train_pred = forest_model.predict(X_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    6.7s
[Parallel(n_jobs=8)]: Done 256 out of 256 | elapsed:    9.2s finished


In [21]:
roc_auc_score(y_train, train_pred)

0.9999993064943936

In [22]:
val_pred = forest_model.predict(X_val)
roc_auc_score(y_val, val_pred)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    2.9s
[Parallel(n_jobs=8)]: Done 256 out of 256 | elapsed:    4.1s finished


0.8610982137932259

In [23]:
test_df = pd.read_csv( F'./data/{feature_set}/test_features.csv')

In [26]:
forest_model.predict(test_df.drop(columns=['TX_ID']))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 256 out of 256 | elapsed:    0.8s finished


array([0.00390625, 0.        , 0.        , ..., 0.01171875, 0.00390625,
       0.00390625])

In [27]:
test_res = forest_model.predict(test_df.drop(columns=['TX_ID']))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 256 out of 256 | elapsed:    0.8s finished


In [ ]:
test_df['TX_FRAUD']  = test_res

In [ ]:
submission_forest = test_df[['TX_ID','TX_FRAUD']]

In [ ]:
test_trans = pd.read_csv( F'./data/transactions_test.csv')
test_trans = test_trans['TX_ID']
submission_forest.set_index('TX_ID',inplace=True)
submission = submission_forest.reindex(index =test_trans)

In [ ]:
location = 'results'
submission.to_csv(F'./{location}/results_5_forest.csv',index=True)